In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')

    !pip install umap-learn
    !pip install umap-learn[plot]
    !pip install holoviews

    !pip install joypy

    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
elif "zmqshell" in str(get_ipython()):
    print("Running on MIRP")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido"
else:
    import pathlib
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    print("Running local")

Running on MIRP


In [2]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from six.moves import xrange
import datetime
import gc
import pandas as pd
import joypy

from scipy import signal
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler as Normalizer

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torchaudio.transforms as audio_transform


#from ResidualStack import ResidualStack
#from Residual import Residual

from Jaguas_DataLoader_rainless import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel
from AE_Clustering import AE_Clustering 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()

from datetime import timedelta
import wandb
from wandb import AlertLevel

!pip install umap-learn
!pip install umap-learn[plot]
import umap
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import seaborn as sns
import pickle as pkl

import random
def _set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).
 
    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
_set_seed(1024)

gc.collect()

/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/umap/dist

35139

In [3]:
model_type = "AE"
identifier = "batch_size_14_num_hiddens_64_"
day = 4
hour = 9
date_format = f"day_{day}_hour_{hour}"

# y = torch.load(f"temporal_zamuro/Features/{folder}/AE_labels_{date_format}_No_rain_Audios_Jaguas.pth",  map_location=torch.device('cpu'))
# X = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/AE_features_Zamuro.pth",  map_location=torch.device('cpu'))
X_ae_norm = np.load(f"temporal_zamuro/Features/X_ae_norm.pth.npy")
y_ae = np.load(f"temporal_zamuro/Features/y_ae.pth.npy")
audios = pd.read_csv(f"Complementary_Files/zamuro_audios.csv", index_col=0)
recorders = pd.read_csv(f"Complementary_Files/zamuro_recorders.csv")
# y_path = torch.load(f"{root}/Zamuro/temporal_zamuro/Features/AE_test_path_samples_Zamuro.pth",  map_location=torch.device('cpu'))
# path_flat = [item for sublist in y_path for item in sublist]
# path_flat = np.asarray(path_flat)



In [4]:
from datetime import datetime, time
def obtener_hora(row):
    hora_str = row['name_FI'].split('_')[1]  # Obtener la parte de la hora
    hora = time(hour=int(hora_str[:2]), minute=int(hora_str[2:4]), second=int(hora_str[4:6]))
    return hora

# Aplicar la función a cada fila y crear una nueva columna 'hora'
audios['hora'] = audios.apply(obtener_hora, axis=1)


In [5]:
from sklearn.preprocessing import LabelEncoder
y_hora = []
y_site = []
y_cover = []
for i in range(len(y_ae)):
    y_hora.append(int(y_ae[i].split("_")[2][0:2]))
    y_site.append(y_ae[i].split("_")[0])
    y_cover.append(recorders[recorders["field_number_PR"] == y_ae[i].split("_")[0]]["cover"].values[0])
    
encoder = LabelEncoder()
y_site = encoder.fit_transform(y_site)
y_cover = encoder.fit_transform(y_cover)

In [6]:
tasa_submuestreo = 0.2

# Obtener el número de muestras a submuestrear
num_muestras_submuestreadas = int(len(X_ae_norm) * tasa_submuestreo)

# Generar índices aleatorios sin reemplazo
indices_submuestreo = np.random.choice(len(X_ae_norm), num_muestras_submuestreadas, replace=False)

X_norm = X_ae_norm[indices_submuestreo]
y_n = [y_cover[i] for i in indices_submuestreo]

In [7]:
def projector(data, y=None, method="PCA", n_components=2, random_state=0, **kwargs):
    
    if method == "PCA":
        X = PCA(n_components=n_components, random_state=0).fit_transform(data)
    elif method == "ICA":
        X = FastICA(n_components=n_components,
                    random_state=0).fit_transform(data)
    elif method == "tSNE":
        X = TSNE(n_components=n_components, learning_rate=kwargs["learning_rate"],
                 init=kwargs["init"], random_state=random_state,
                 n_jobs=kwargs["n_jobs"]).fit_transform(X_PCA)
        
    elif method == "UMAP":
        reducer = umap.UMAP(n_components=n_components, min_dist=kwargs["min_dist"],
                            metric=kwargs["metric"], n_neighbors=kwargs["n_neighbors"],
                            random_state=random_state, n_jobs=kwargs["n_jobs"])
        X = reducer.fit_transform(data)
    return X

### UMAP grid search
- https://pair-code.github.io/understanding-umap/
- Pre-selected params are: 0.1-60, 0.5-60, 0.5-180 and 1-60

In [ ]:
etiquetas = y_n
recuento_etiquetas = {}
for etiqueta in etiquetas:
    if etiqueta in recuento_etiquetas:
        recuento_etiquetas[etiqueta] += 1
    else:
        recuento_etiquetas[etiqueta] = 1

# Mostramos el recuento de cada etiqueta
for etiqueta, recuento in recuento_etiquetas.items():
    print(f"Etiqueta '{etiqueta}': {recuento} muestras")

In [ ]:
min_dist = 0.1
n_neighbor = 180
metric = "euclidean"
n_components = [2, 15, 30, 45, 60]

for i,component in enumerate(n_components):
    print(f"Running {component} components. Iteration {i+1} of {len(n_components)}")
    X_projected = projector(X_ae_norm, method="UMAP", 
                                 min_dist=min_dist,
                                 n_neighbors=n_neighbor,
                                 metric=metric, n_components=component, n_jobs=-1)

    np.save(f'temporal_zamuro/Features/X_ae_norm_UMAP_full_{component}D.npy', X_projected)

### Interactive 2D

In [ ]:
import plotly.graph_objs as go
import plotly.io as pio


# Crear la figura interactiva
fig = go.Figure()

# Agregar los puntos a la figura para cada par de componentes
for i in range(0, 30, 2):
    fig.add_trace(go.Scatter(
        x=X_projected_30d[:, i],
        y=X_projected_30d[:, i+1],
        mode='markers',
        marker=dict(
            size=1,
            color=y_n,
            colorscale='Viridis',
            opacity=1
        ),
        name=f'Componentes {i+1}-{i+2}'
    ))

# Establecer el diseño del gráfico
fig.update_layout(
    xaxis_title='UMAP Component 1',
    yaxis_title='UMAP Component 2',
    title='UMAP Projection in 2D with All Component Pairs (30 components)'
)

# Guardar la figura como un archivo HTML interactivo
# pio.write_html(fig, 'umap_projection_2d_all_pairs_30_components_interactive.html')

# Mostrar la figura interactiva
fig.show()

### Interactive 3D

In [ ]:
import plotly.graph_objs as go
import plotly.io as pio

min_dist = 0.5
n_neighbor = 180
metric = "euclidean"


# Crear la figura interactiva
fig = go.Figure()

# Agregar los puntos a la figura
for i in range(0, 30, 3):
    fig.add_trace(go.Scatter3d(
        x=X_projected_30d[:, i],
        y=X_projected_30d[:, i+1],
        z=X_projected_30d[:, i+2],
        mode='markers',
        marker=dict(
            size=1,
            color=y_n,
            colorscale='Viridis',
            opacity=0.5
        ),
        name=f'Componentes {i+1}-{i+3}'
    ))

# Establecer el diseño de la escena
fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='UMAP Projection in 3D with 30 components'
)

# Guardar la figura como un archivo HTML interactivo
pio.write_html(fig, 'umap_projection_3d_18_components_interactive5.html')

# Mostrar la figura interactiva
fig.show()


In [ ]:
metrics = ["euclidean", "minkowski", "correlation"]
min_dists = [0.1, 0.5, 1]
n_neighbors = [15, 60, 180, 512]

label_mapping = {valor: f"Clase {valor}" for valor in np.unique(y_n)}

for metric in metrics:
    for min_dist in min_dists:
        for n_neighbor in n_neighbors:
            print(f"iterating with: \n metric: {metric} \n min_dist: {min_dist}, \n n_neighbors: {n_neighbor}")
            X_projected = projector(X_ae_norm, method="UMAP", 
                                    min_dist=min_dist,
                                    n_neighbors=n_neighbor,
                                    metric=metric, n_jobs=-1)
            labels = [label_mapping[valor] for valor in y_n]
            plt.scatter(X_projected[:,0], X_projected[:,1],alpha=0.5, s=0.1, c=y_n, label=labels)
#             plt.legend()
#             plt.savefig(f"temporal_zamuro/zamuro_projections/hour/UMAP_metric-{metric}_mindist-{min_dist}_nneighbors-{n_neighbor}.pdf")
            plt.show()
       

### Connectivity

In [ ]:
mapper = umap.UMAP(n_components=2, min_dist=0.1,
                            metric="euclidean", n_neighbors=256,
                            random_state=0, n_jobs=-1).fit(X_norm)
X_projected = mapper.transform(X_norm)

In [ ]:
import umap.plot
umap.plot.connectivity(mapper, show_points=True, edge_bundling='hammer')

In [ ]:
import umap.plot
metrics = ["euclidean"]
min_dists = [0.01, 0.05, 0.1]
n_neighbors = [180]

labels = np.asarray(y_n)

for metric in metrics:
    for min_dist in min_dists:
        for n_neighbor in n_neighbors:
            print(f"iterating with: \n metric: {metric} \n min_dist: {min_dist}, \n n_neighbors: {n_neighbor}")
            mapper = umap.UMAP(n_components=2, min_dist=min_dist,
                            metric=metric, n_neighbors=n_neighbor,
                            random_state=0, n_jobs=-1).fit(X_norm)
            X_projected = mapper.transform(X_norm)
            y_n = [y_hora[i] for i in indices_submuestreo]
            labels = np.asarray(y_n)
            umap.plot.connectivity(mapper, show_points=True, labels=labels, theme= "viridis", edge_bundling='hammer')
            plt.savefig(f"temporal_zamuro/zamuro_projections/connectivity/hour/UMAP_connectivity_metric-{metric}_mindist-{min_dist}_nneighbors-{n_neighbor}_hammer.pdf")
            plt.show()            
            y_n = [y_site[i] for i in indices_submuestreo]
            labels = np.asarray(y_n)
            umap.plot.connectivity(mapper, show_points=True, labels=labels, theme= "viridis", edge_bundling='hammer')
            plt.savefig(f"temporal_zamuro/zamuro_projections/connectivity/site/UMAP_connectivity_metric-{metric}_mindist-{min_dist}_nneighbors-{n_neighbor}_hammer.pdf")
            plt.show()            
            y_n = [y_cover[i] for i in indices_submuestreo]
            labels = np.asarray(y_n)
            umap.plot.connectivity(mapper, show_points=True, labels=labels, theme= "viridis", edge_bundling='hammer')
            plt.savefig(f"temporal_zamuro/zamuro_projections/connectivity/cover/UMAP_connectivity_metric-{metric}_mindist-{min_dist}_nneighbors-{n_neighbor}_hammer.pdf")
            plt.show()

In [ ]:
import numpy as np
import umap.plot

for i in range(10):
    selected_labels = np.random.choice(np.unique(y_site), size=10, replace=False)

    selected_indices = np.where(np.isin(y_site, selected_labels))[0]

    sample_size = int(1 * len(selected_indices))

    subsample_indices = np.random.choice(selected_indices, size=sample_size, replace=False)

    # Submuestrear los datos
    X_norm = X_ae_norm[subsample_indices]
    y_site_subsampled = y_site[subsample_indices]

    mapper = umap.UMAP(n_components=2, min_dist=0.5,
                                metric="euclidean", n_neighbors=180,
                                random_state=0, n_jobs=-1).fit(X_norm, y=y_site_subsampled)
    X_projected = mapper.transform(X_norm)
    y_n = y_site_subsampled
    labels = np.asarray(y_n)
    umap.plot.connectivity(mapper, show_points=True, labels=labels, theme= "viridis")
    plt.savefig(f"temporal_zamuro/zamuro_projections/connectivity/site_subsampled_guided/UMAP_connectivity_metric-{mapper.metric}_mindist-{mapper.min_dist}_nneighbors-{mapper.n_neighbors}_{i}.pdf")
    plt.show()
    umap.plot.points(mapper, labels=labels, theme= "viridis", alpha=0.2 )
    plt.savefig(f"temporal_zamuro/zamuro_projections/connectivity/site_subsampled_guided/UMAP_points_metric-{mapper.metric}_mindist-{mapper.min_dist}_nneighbors-{mapper.n_neighbors}_{i}.pdf")
    plt.show()


In [ ]:
# Submuestrear los datos
X_norm = X_ae_norm[subsample_indices]
y_site_subsampled = y_site[subsample_indices]

mapper = umap.UMAP(n_components=2, min_dist=0.1,
                            metric="euclidean", n_neighbors=180,
                            random_state=0, n_jobs=-1).fit(X_norm)
X_projected = mapper.transform(X_norm)
y_n = y_site_subsampled
labels = np.asarray(y_n)
umap.plot.connectivity(mapper, show_points=True, labels=labels, theme= "viridis")
plt.savefig(f"temporal_zamuro/zamuro_projections/connectivity/hour/UMAP_connectivity_metric-{metric}_mindist-{min_dist}_nneighbors-{n_neighbor}_hammer.pdf")
plt.show()    


In [ ]:
mapper.
#https://www.researchgate.net/publication/353863256_Clustering_with_UMAP_Why_and_How_Connectivity_Matters

In [ ]:
mapper = umap.UMAP(n_components=2, min_dist=0.6,
                            metric="euclidean", n_neighbors=180,
                            random_state=0, n_jobs=-1).fit(X_norm, y=y_n)
umap.plot.points(mapper, labels=labels, theme= "viridis", alpha=0.2 )
umap.plot.connectivity(mapper, show_points=True, labels=labels, theme= "viridis")


In [ ]:
import umap.plot
y_n = [y_cover[i] for i in indices_submuestreo]
labels = np.asarray(y_n)
umap.plot.points(mapper, labels=labels, theme= "viridis")
plt.savefig("pruebaaa_cover2")

In [ ]:
import umap.plot
umap.plot.connectivity(mapper, show_points=True, labels=labels, theme="viridis")
plt.savefig("connectivity_pruebaaa_sites2")

### PACMAP

In [ ]:
import pacmap
import time
FP_ratios = [0.5, 1, 5, 10, 20, 50]
MN_ratios = [0.1, 0.3, 0.5, 0.7, 1]
n_neighbors = [15, 60, 180]
# y_n = [y_cover[i] for i in indices_submuestreo]
exec_times = []
for FP_ratio in FP_ratios:
    for MN_ratio in MN_ratios:
        for n_neighbor in n_neighbors:
            init_time = time.time()
            print(f"Running FP Ratio: {FP_ratio}, MN Ratio: {MN_ratio}, n_neighbors: {n_neighbor}")
            embedded = pacmap.PaCMAP(n_components=2, n_neighbors=n_neighbor, MN_ratio=MN_ratio, FP_ratio=FP_ratio) 
            X_pacmap = embedded.fit_transform(X_norm, init="pca")
            fig, ax = plt.subplots(1, 1, figsize=(6, 6))
            ax.scatter(X_pacmap[:, 0], X_pacmap[:, 1], cmap="viridis", s=0.01, alpha=0.5)
            plt.savefig(f"temporal_zamuro/zamuro_projections/PACMAP/PACMAP_MNratio-{MN_ratio}_nneighbors-{n_neighbor}_FPratio_{FP_ratio}.pdf")
            plt.title(f"PACMAP_MNratio-{MN_ratio}_nneighbors-{n_neighbor}_FPratio_{FP_ratio}")
            plt.show()
            end_time = time.time()
            exec_time = end_time - init_time
            exec_times.append(exec_time)
            print(f"Time of execution: {exec_time}")

In [ ]:
embedded = pacmap.PaCMAP(n_components=2, n_neighbors=60, MN_ratio=1, FP_ratio=20) 
X_pacmap = embedded.fit_transform(X_norm, init="pca")

In [ ]:
embedded = pacmap.PaCMAP(n_components=2, n_neighbors=n_neighbor, MN_ratio=MN_ratio, FP_ratio=FP_ratio) 

In [ ]:
np.save(f'temporal_zamuro/zamuro_projections/PACMAP/exec_times.npy', X_projected)

### PCA Elbow

In [ ]:
pca = PCA(n_components = 5184)
pca_fit = pca.fit(X_ae_norm)

In [ ]:
import seaborn as sns

# Set style and figure size
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))

# Scree Plot
PC_values = np.arange(pca.n_components) + 1
plt.plot(PC_values[0:80], pca.explained_variance_ratio_[0:80].cumsum(),
         'o-',markersize=4, linewidth=1, color='maroon')

# Add text with explained variance ratio values on selected points
points_to_annotate = [10, 20, 30, 40, 50, 60, 80]  # Add values for the points you want to annotate
variance_ratio = pca.explained_variance_ratio_.cumsum()
for point in points_to_annotate:
    plt.text(point, variance_ratio[point-1]+0.02, f'{variance_ratio[point-1]*100:.2f}%', ha='center', va='bottom', fontsize=10)

# Titles and axis labels
plt.title('Scree Plot', fontsize=16)
plt.xlabel('Number of Principal Components', fontsize=14)
plt.ylabel('Explained Variance', fontsize=14)

# Save the figure as a PDF file
plt.savefig("temporal_zamuro/zamuro_projections/PCA/scree_plot.pdf",
             dpi=300)

# Show the plot
plt.show()

